In [1]:
# !pip install pytorch-tabnet

In [2]:
import torch
import optuna
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


import warnings
warnings.filterwarnings("ignore")

In [3]:
# 데이터 불러오기
train_data = pd.read_csv("~/project/MIMIC-III/Data/FINAL/Final Data_Split/train_data(50%)_down.csv")
test_data = pd.read_csv("~/project/MIMIC-III/Data/FINAL/Final Data_Split/test_data(50%)_down.csv")

train_data.drop(["SUBJECT_ID","HADM_ID", "DOA", "TLOS", "ETHNICITY"], axis=1, inplace=True)
test_data.drop(["SUBJECT_ID","HADM_ID", "DOA", "TLOS", "ETHNICITY"], axis=1, inplace=True)

train_data = pd.get_dummies(train_data, columns=["GENDER"])
test_data = pd.get_dummies(test_data, columns=["GENDER"])

# train_data와 test_data의 칼럼을 맞추기 위해 차집합을 계산
missing_columns_in_test = set(train_data.columns) - set(test_data.columns)
missing_columns_in_train = set(test_data.columns) - set(train_data.columns)

# test_data에 train_data의 더미 변수를 추가하고 0으로 채워주기
for col in missing_columns_in_test:
    test_data[col] = 0

# train_data에 test_data의 더미 변수를 추가하고 0으로 채워주기
for col in missing_columns_in_train:
    train_data[col] = 0
    

# 데이터 분할 결과 확인
print("Train Data:")
print(train_data['y'].value_counts())
print("Test Data:")
print(test_data['y'].value_counts())

X_train = train_data.drop(['y'], axis=1).values
y_train = train_data['y'].values

X_test = test_data.drop(['y'], axis=1).values
y_test = test_data['y'].values

Train Data:
0    489
1    489
Name: y, dtype: int64
Test Data:
0    341
1    116
Name: y, dtype: int64


In [4]:
from sklearn.metrics import SCORERS

print(SCORERS.keys())


dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [5]:
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
auroc_list = []

# TabNetClassifier의 하이퍼파라미터 그리드를 정의합니다.
param_dist = {
    'n_d': list(range(2, 65)),  # Decision steps
    'n_a': list(range(2, 65)),  # Attention steps
    'n_steps': list(range(2, 33)),  # Number of steps in the architecture
    'gamma': [1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0],  # Regularization factor
    'lambda_sparse': [0.001, 0.01, 0.1, 1]  # Sparsity penalty
}

# 10번 반복
for i in range(10):
    
    # TabNetClassifier 모델을 초기화합니다.
    clf = TabNetClassifier()
    
   # # RandomizedSearchCV를 사용하여 하이퍼파라미터 튜닝을 수행합니다.
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=10, cv=3, scoring='f1_weighted', n_jobs=-1)
    random_search.fit(X_train, y_train)

    # 최적의 하이퍼파라미터 조합을 출력합니다.
    # print("Best Parameters: ", random_search.best_params_)

    # 테스트 데이터에 대한 예측을 수행합니다.
    y_pred = random_search.predict(X_test)
    y_score = random_search.predict_proba(X_test)[:, 1]
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_score)

    rounded_accuracy = round(accuracy * 100, 2)
    rounded_precision = round(precision * 100, 2)
    rounded_recall = round(recall * 100, 2)
    rounded_f1 = round(f1 * 100, 2)
    rounded_auroc = round(auroc * 100, 2)
    
    # 결과를 리스트에 추가
    accuracy_list.append(rounded_accuracy)
    precision_list.append(rounded_precision)
    recall_list.append(rounded_recall)
    f1_list.append(rounded_f1)
    auroc_list.append(rounded_auroc)

    # 매 반복마다 결과 출력
    print(f"Iteration {i + 1}:")
    print("Test Accuracy:", rounded_accuracy)
    print("Precision:", rounded_precision)
    print("Recall:", rounded_recall)
    print("F1-score:", rounded_f1)
    print("AUROC:", rounded_auroc)

/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_

epoch 0  | loss: 0.0     |  0:00:00s
epoch 1  | loss: 0.0     |  0:00:00s
epoch 2  | loss: 0.0     |  0:00:00s
epoch 3  | loss: 0.0     |  0:00:00s
epoch 4  | loss: 0.0     |  0:00:00s
epoch 5  | loss: 0.0     |  0:00:00s
epoch 6  | loss: 0.0     |  0:00:00s
epoch 7  | loss: 0.0     |  0:00:00s
epoch 8  | loss: 0.0     |  0:00:00s
epoch 9  | loss: 0.0     |  0:00:00s
epoch 10 | loss: 0.0     |  0:00:00s
epoch 11 | loss: 0.0     |  0:00:00s
epoch 12 | loss: 0.0     |  0:00:00s
epoch 13 | loss: 0.0     |  0:00:00s
epoch 14 | loss: 0.0     |  0:00:00s
epoch 15 | loss: 0.0     |  0:00:00s
epoch 16 | loss: 0.0     |  0:00:00s
epoch 17 | loss: 0.0     |  0:00:00s
epoch 18 | loss: 0.0     |  0:00:00s
epoch 19 | loss: 0.0     |  0:00:00s
epoch 20 | loss: 0.0     |  0:00:00s
epoch 21 | loss: 0.0     |  0:00:00s
epoch 22 | loss: 0.0     |  0:00:00s
epoch 23 | loss: 0.0     |  0:00:00s
epoch 24 | loss: 0.0     |  0:00:00s
epoch 25 | loss: 0.0     |  0:00:00s
epoch 26 | loss: 0.0     |  0:00:00s
e

/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_

epoch 0  | loss: 0.0     |  0:00:00s
epoch 1  | loss: 0.0     |  0:00:00s
epoch 2  | loss: 0.0     |  0:00:00s
epoch 3  | loss: 0.0     |  0:00:00s
epoch 4  | loss: 0.0     |  0:00:00s
epoch 5  | loss: 0.0     |  0:00:00s
epoch 6  | loss: 0.0     |  0:00:00s
epoch 7  | loss: 0.0     |  0:00:00s
epoch 8  | loss: 0.0     |  0:00:00s
epoch 9  | loss: 0.0     |  0:00:00s
epoch 10 | loss: 0.0     |  0:00:00s
epoch 11 | loss: 0.0     |  0:00:00s
epoch 12 | loss: 0.0     |  0:00:00s
epoch 13 | loss: 0.0     |  0:00:00s
epoch 14 | loss: 0.0     |  0:00:00s
epoch 15 | loss: 0.0     |  0:00:00s
epoch 16 | loss: 0.0     |  0:00:00s
epoch 17 | loss: 0.0     |  0:00:00s
epoch 18 | loss: 0.0     |  0:00:00s
epoch 19 | loss: 0.0     |  0:00:00s
epoch 20 | loss: 0.0     |  0:00:00s
epoch 21 | loss: 0.0     |  0:00:00s
epoch 22 | loss: 0.0     |  0:00:00s
epoch 23 | loss: 0.0     |  0:00:00s
epoch 24 | loss: 0.0     |  0:00:00s
epoch 25 | loss: 0.0     |  0:00:00s
epoch 26 | loss: 0.0     |  0:00:00s
e

/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_

...

epoch 63 | loss: 0.0     |  0:00:00s
epoch 64 | loss: 0.0     |  0:00:00s
epoch 65 | loss: 0.0     |  0:00:00s
epoch 66 | loss: 0.0     |  0:00:00s
epoch 67 | loss: 0.0     |  0:00:00s
epoch 68 | loss: 0.0     |  0:00:00s
epoch 69 | loss: 0.0     |  0:00:00s
epoch 70 | loss: 0.0     |  0:00:00s
epoch 71 | loss: 0.0     |  0:00:00s
epoch 72 | loss: 0.0     |  0:00:00s
epoch 73 | loss: 0.0     |  0:00:00s
epoch 74 | loss: 0.0     |  0:00:00s
epoch 75 | loss: 0.0     |  0:00:00s
epoch 76 | loss: 0.0     |  0:00:00s
epoch 77 | loss: 0.0     |  0:00:00s
epoch 78 | loss: 0.0     |  0:00:00s
epoch 79 | loss: 0.0     |  0:00:00s
epoch 80 | loss: 0.0     |  0:00:00s
epoch 81 | loss: 0.0     |  0:00:00s
epoch 82 | loss: 0.0     |  0:00:00s
epoch 83 | loss: 0.0     |  0:00:00s
epoch 84 | loss: 0.0     |  0:00:00s
epoch 85 | loss: 0.0     |  0:00:00s
epoch 86 | loss: 0.0     |  0:00:00s
epoch 87 | loss: 0.0     |  0:00:00s
epoch 88 | loss: 0.0     |  0:00:00s
epoch 89 | loss: 0.0     |  0:00:00s
e

/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_

epoch 63 | loss: 0.0     |  0:00:00s
epoch 64 | loss: 0.0     |  0:00:00s
epoch 65 | loss: 0.0     |  0:00:00s
epoch 66 | loss: 0.0     |  0:00:00s
epoch 67 | loss: 0.0     |  0:00:00s
epoch 68 | loss: 0.0     |  0:00:00s
epoch 69 | loss: 0.0     |  0:00:00s
epoch 70 | loss: 0.0     |  0:00:00s
epoch 71 | loss: 0.0     |  0:00:00s
epoch 72 | loss: 0.0     |  0:00:00s
epoch 73 | loss: 0.0     |  0:00:00s
epoch 74 | loss: 0.0     |  0:00:00s
epoch 75 | loss: 0.0     |  0:00:00s
epoch 76 | loss: 0.0     |  0:00:00s
epoch 77 | loss: 0.0     |  0:00:00s
epoch 78 | loss: 0.0     |  0:00:00s
epoch 79 | loss: 0.0     |  0:00:00s
epoch 80 | loss: 0.0     |  0:00:00s
epoch 81 | loss: 0.0     |  0:00:00s
epoch 82 | loss: 0.0     |  0:00:00s
epoch 83 | loss: 0.0     |  0:00:00s
epoch 84 | loss: 0.0     |  0:00:00s
epoch 85 | loss: 0.0     |  0:00:00s
epoch 86 | loss: 0.0     |  0:00:00s
epoch 87 | loss: 0.0     |  0:00:00s
epoch 88 | loss: 0.0     |  0:00:00s
epoch 89 | loss: 0.0     |  0:00:00s
e

/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/watercar99/.local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_

epoch 42 | loss: 0.0     |  0:00:00s
epoch 43 | loss: 0.0     |  0:00:00s
epoch 44 | loss: 0.0     |  0:00:00s
epoch 45 | loss: 0.0     |  0:00:00s
epoch 46 | loss: 0.0     |  0:00:00s
epoch 47 | loss: 0.0     |  0:00:00s
epoch 48 | loss: 0.0     |  0:00:00s
epoch 49 | loss: 0.0     |  0:00:00s
epoch 50 | loss: 0.0     |  0:00:00s
epoch 51 | loss: 0.0     |  0:00:00s
epoch 52 | loss: 0.0     |  0:00:00s
epoch 53 | loss: 0.0     |  0:00:00s
epoch 54 | loss: 0.0     |  0:00:00s
epoch 55 | loss: 0.0     |  0:00:00s
epoch 56 | loss: 0.0     |  0:00:00s
epoch 57 | loss: 0.0     |  0:00:00s
epoch 58 | loss: 0.0     |  0:00:00s
epoch 59 | loss: 0.0     |  0:00:00s
epoch 60 | loss: 0.0     |  0:00:00s
epoch 61 | loss: 0.0     |  0:00:00s
epoch 62 | loss: 0.0     |  0:00:00s
epoch 63 | loss: 0.0     |  0:00:00s
epoch 64 | loss: 0.0     |  0:00:00s
epoch 65 | loss: 0.0     |  0:00:00s
epoch 66 | loss: 0.0     |  0:00:00s
epoch 67 | loss: 0.0     |  0:00:00s
epoch 68 | loss: 0.0     |  0:00:00s
e

In [6]:
data = {
    'Accuracy': accuracy_list,
    'Precision': precision_list,
    'Recall': recall_list,
    'F1 Score': f1_list,
    'AUROC' : auroc_list,
    # 'Predictions': y_pred_list
}

# 딕셔너리를 DataFrame으로 변환
df = pd.DataFrame(data)
df

,Accuracy,Precision,Recall,F1 Score,AUROC
0,61.71,27.13,30.17,28.57,51.24
1,52.30,25.24,44.83,32.30,48.66
2,49.02,27.24,60.34,37.53,51.67
3,47.05,27.17,64.66,38.27,50.70
4,49.67,23.11,42.24,29.88,47.05
5,43.54,24.46,58.62,34.52,48.50
6,56.67,26.44,39.66,31.72,51.65
7,61.71,27.82,31.90,29.72,51.70
8,46.61,25.00,55.17,34.41,49.21
9,56.89,25.45,36.21,29.89,50.30


In [7]:
# DataFrame을 CSV 파일로 저장
df.to_csv('~/project/MIMIC-III/Model/Output/TabNet/TN(50%).csv', index=False)  # index를 저장하지 않으려면 index=False로 설정
